In [2]:
#!/usr/bin/env python3
"""Show a text-mode spectrogram using live microphone data."""
import argparse
import logging
import numpy as np
import time
import soundfile
import itertools
import IPython.display as ipd
import matplotlib.pyplot as plt
import os
import random
import sounddevice as sd
import audioop
import librosa
#import webrtcvad as wrt
import struct

!pip install soundfile
!pip install sounddevice
!pip install librosa

In [3]:
DURATION=20
gain=10
range=[100,2000]
high=2000
low=100
screenwidth=79

In [22]:
WORD_TO_TRAIN = ['cipi','bangun']
PATH_TO_TRAIN = os.getcwd()+'/DeadSimpleSpeechRecognizer/newdata/train'
PATH_TO_TEST = os.getcwd()+'/DeadSimpleSpeechRecognizer/newdata/test'
FILE_PREFIX = "irza"
WORKPATH = PATH_TO_TRAIN

magnitudo=[]
audiodata=[]
recording=False
end_count = 0
start_count = 0
def record(length=1, filename=None, thres=0):
    """ 
    Merekam suara secara stream dan metode callback
    """

    global cumulated_status, audiodata, magnitudo, end_count, recording, start_count
    try:
        import sounddevice as sd

        #samplerate = sd.query_devices(args.device, 'input')['default_samplerate']
        samplerate = 16000.0

        delta_f = (high - low) / screenwidth
        fftsize = np.ceil(samplerate / delta_f).astype(int)
        low_bin = int(np.floor(low / delta_f))

        cumulated_status = sd.CallbackFlags()

        def callback(indata, frames, time, status):
            global cumulated_status, recording, end_count, audiodata, magnitudo, start_count
            
            
            cumulated_status |= status
            if any(indata):
                
                #chv = wrt.Vad()
                #if chv.is_speech(b''.join(indata.tolist()),int(samplerate)): print("voice")
                
                
                magnitude = np.abs(np.fft.rfft(indata[:, 0], n=fftsize))
                magnitude *= gain / fftsize

                rms = librosa.feature.rmse(S=indata)
                rms = int(rms*32768)
                start_count += 1
                if rms>=thres:
                    end_count = 0
                    if not recording and (start_count > 20):
                            audiodata=[]
                            magnitudo=[]
                            recording = True
                            print('O', end='', flush=True)
                            audiodata.extend(itertools.chain(indata.tolist()))
                            magnitudo.append(magnitude)

                    else:
                        print('x', end='', flush=True)
                        #add to list
                        audiodata.extend(itertools.chain(indata.tolist()))
                        magnitudo.append(magnitude)
                else:
                    if recording:
                        audiodata.extend(itertools.chain(indata.tolist()))
                        magnitudo.append(magnitude)

                    if (end_count > 20) and recording:
                        recording=False
                        end_count = 0
                        print('X', end='', flush=True)
                        start_count = 0
                    else:
                        print('.', end='', flush=True)
                        end_count += 1


        with sd.InputStream(device=None, channels=1, callback=callback,
                            blocksize=int(samplerate * DURATION / 1000),
                            samplerate=samplerate):
            while True:
                #response = input()
                #if response in ('', 'q', 'Q'):
                time.sleep(length)
                break
            if filename!=None: soundfile.write(filename,audiodata,16000)

        if cumulated_status:
            logging.warning(str(cumulated_status))
    except Exception as e:
        print(e)
        
def auplot(data=None):
    if data==None: data = audiodata
    plt.plot(np.array(data))

def generate(WORKPATH=WORKPATH,style="FULL", frame=5, thres=0):
    """ 
    Gunakan generate ini untuk membuat SET dari yang ingin di training atau
    untuk SET TESTING, gunakan frame untuk menandakan panjang dari frame
    yang ingin di deteksi
    """
    files=[]
    #Generate AUDIO FILE

    #create the dir 
    for i in WORD_TO_TRAIN:
        try:
            os.mkdir(PATH_TO_TRAIN+"/"+i)
            os.mkdir(PATH_TO_TEST+"/"+i)
        except:
            pass
    
    rann = str(random.randint(0,5000))
    print("Sequence:",rann)

    #use this for shuffle the word
    random.shuffle(WORD_TO_TRAIN)

    for i in WORD_TO_TRAIN:
        #gunakan PATH_TO_TEST atau PATH_TO_TRAIN
        file = WORKPATH+"/"+i+"/"+i+"-"+FILE_PREFIX+"-"+rann+".wav"
        
        print("\nSilahkan baca kalau sudah recording :",i)
        record(2,filename=file,thres=thres)
        files.append(file)
    
    print("Selesai proses merekam ke:",WORKPATH)
    return files

def del_small_file(minsize=40000):
    for i in WORD_TO_TRAIN:
        f = WORKPATH+"/"+i
        print(f)
        lst = os.listdir(f)
        for s in lst:
            fl = str(f+'/'+s)
            sz = os.path.getsize(fl)
            if sz<minsize:
                os.remove(fl)
                print(fl,sz)  

In [23]:
def record(length=1, reclength=1, filename=None, thres=0):
    """ 
    Merekam suara secara stream dan metode callback
    """

    global cumulated_status, end_count, start_count, recording, magnitudo, audiodata
    end_count=False
    start_count = 0
    recording=False
    magnitudo=[]
    audiodata=[]
    try:
        import sounddevice as sd

        #samplerate = sd.query_devices(args.device, 'input')['default_samplerate']
        samplerate = 16000.0

        delta_f = (high - low) / screenwidth
        fftsize = np.ceil(samplerate / delta_f).astype(int)
        low_bin = int(np.floor(low / delta_f))

        cumulated_status = sd.CallbackFlags()

        def callback(indata, frames, time, status):
            global cumulated_status, audiodata, magnitudo, end_count, start_count, recording
            
            
            cumulated_status |= status
            if any(indata):
                magnitude = np.abs(np.fft.rfft(indata[:, 0], n=fftsize))
                magnitude *= gain / fftsize

                rms = librosa.feature.rmse(S=indata)
                rms = int(rms*32768)
                start_count += 1
                if rms>=thres:
                    if not recording and not end_count:
                        print("Start record")
                        recording = True
                        start_count = 0
                        
                if recording:
                    audiodata.extend(itertools.chain(indata.tolist()))
                    magnitudo.append(magnitude)
                    if start_count == int(samplerate / (samplerate * DURATION / 1000)):
                        print("End record")
                        end_count=True
                        recording=False


        with sd.InputStream(device=None, channels=1, callback=callback,
                            blocksize=int(samplerate * DURATION / 1000),
                            samplerate=samplerate):
            while True:
                #response = input()
                #if response in ('', 'q', 'Q'):
                time.sleep(length)
                break
            if filename!=None: soundfile.write(filename,audiodata,16000)

        if cumulated_status:
            logging.warning(str(cumulated_status))
    except Exception as e:
        print(e)
        
def auplot(data=None):
    if data==None: data = audiodata
    plt.plot(np.array(data))

def generate(WORKPATH=WORKPATH,style="FULL", frame=5, thres=0):
    """ 
    Gunakan generate ini untuk membuat SET dari yang ingin di training atau
    untuk SET TESTING, gunakan frame untuk menandakan panjang dari frame
    yang ingin di deteksi
    """
    files=[]
    #Generate AUDIO FILE

    #create the dir 
    for i in WORD_TO_TRAIN:
        try:
            os.mkdir(PATH_TO_TRAIN+"/"+i)
            os.mkdir(PATH_TO_TEST+"/"+i)
        except:
            pass
    
    rann = str(random.randint(0,5000))
    print("Sequence:",rann)

    #use this for shuffle the word
    random.shuffle(WORD_TO_TRAIN)

    for i in WORD_TO_TRAIN:
        #gunakan PATH_TO_TEST atau PATH_TO_TRAIN
        file = WORKPATH+"/"+i+"/"+i+"-"+FILE_PREFIX+"-"+rann+".wav"
        
        print("\nSilahkan baca kalau sudah recording :",i)
        record(2,filename=file,thres=thres)
        files.append(file)
    
    print("Selesai proses merekam ke:",WORKPATH)
    return files

def del_small_file(minsize=40000):
    for i in WORD_TO_TRAIN:
        f = WORKPATH+"/"+i
        print(f)
        lst = os.listdir(f)
        for s in lst:
            fl = str(f+'/'+s)
            sz = os.path.getsize(fl)
            if sz<minsize:
                os.remove(fl)
                print(fl,sz)  

In [41]:
files=generate(thres=800)
#record(length=3, filename="record.wav", thres=800)

Sequence: 2009

Silahkan baca kalau sudah recording : cipi
Start record
End record

Silahkan baca kalau sudah recording : bangun
Start record
Selesai proses merekam ke: E:\data\python\3.6\Words\chatbot/DeadSimpleSpeechRecognizer/newdata/train


In [ ]:
ipd.Audio('word/train/cipi/cipi-irza-4229.wav')

In [ ]:
ff, sr = librosa.load('word/train/cipi/cipi-irza-2379.wav')

In [1]:
from data.lihat_gambar import lihat_gambar

In [2]:
lihat_gambar.inputx("Hello")

Ini adalah fungsi hebad dari rontok
langsung dari input: Hello


In [43]:
wave, sr = librosa.load('word/train/cipi/cipi-irza-1028-ns-2264.wav', mono=True, sr=None)

4582

In [6]:
lihat_gambar.input("helo")

TypeError: input() missing 1 required positional argument: 'sentence'